In [1]:
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt
import uproot, os, sys
import awkward as ak
# Get the notebook directory
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
# Add the project root to sys.path
sys.path.append(os.path.join(notebook_dir, ".."))
from utils.branches import get_branches
from utils.plot import plot_data
from utils.constants import trigcut, truthpkk
from utils.data_loader import load_mc_data
from matplotlib import rcParams
import matplotlib as mpl
plt.style.use(hep.style.LHCb1)
config = {"mathtext.fontset":'stix'}
rcParams.update(config)

In [2]:
plt.rcParams.update({
    # Keep the font family settings for LHCb style
    "font.family": "serif",
    "font.serif": ["Times", "Computer Modern Roman", "DejaVu Serif"],
    
    # # Increase only the size-related parameters
    # "figure.figsize": (15, 10),  # Larger figure
    # "figure.dpi": 100,          # Screen display
    # "savefig.dpi": 300,         # Saved figure resolution
    
    # # # Increase font sizes while keeping LHCb style
    "font.size": 12,            # Base font size (increase from default)
    "axes.titlesize": 12,       # Title size
    "axes.labelsize": 10,       # Axis label size
    "xtick.labelsize": 12,      # X tick label size
    "ytick.labelsize": 12,      # Y tick label size
    "legend.fontsize": 12       # Legend font size
})


In [3]:


_mc_dd = load_mc_data(
    mc_path="/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC",
    decay_mode="L0barPKpKm",
    particles=["h1", "h2", "p"],
    additional_branches=[
        "p_MC15TuneV1_ProbNNp",
        "h1_MC15TuneV1_ProbNNk",
        "h2_MC15TuneV1_ProbNNk",
        "Bu_TRUEID"
    ],
    tracks=["DD"], # ["DD", "LL"],
    cuts=trigcut + truthpkk
)



_mc_ll = load_mc_data(
    mc_path="/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC",
    decay_mode="L0barPKpKm",
    particles=["h1", "h2", "p"],
    additional_branches=[
        "p_MC15TuneV1_ProbNNp",
        "h1_MC15TuneV1_ProbNNk",
        "h2_MC15TuneV1_ProbNNk",
        "Bu_TRUEID"
    ],
    tracks=["LL"], # ["DD", "LL"],
    cuts=trigcut + truthpkk
)

MC Files being processed with trees ['DD']: ['/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC16MDBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC16MUBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC17MDBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC17MUBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC18MDBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree', '/share/lazy/Mohamed/Bu2LambdaPPP/MC/DaVinciTuples/restripped.MC/MC18MUBu2L0barPKpKm.root:B2L0barPKpKm_DD/DecayTree']
MC Branches being read: ['h1_P', 'h1_PT', 'h1_PE', 'h1_PX', 'h1_PY', 'h1_PZ', 'h1_ID', 'h1_TRACK_Type', 'h1_IPCHI2_OWNPV', 'h2_P', 'h2_PT', 'h2_PE', 'h2_PX', 'h2_PY', 'h2_PZ', 'h2_ID', 'h2_TRACK_Type', 'h2_IPCHI2_OWNPV', 'p_P', 'p_PT', 'p_PE', 'p_

In [4]:
import numpy as np
import awkward as ak
from collections import OrderedDict

def apply_selection_cuts(events, track_type='LL'):
    """
    Apply selection cuts to B+ → Λ0 h1 h2 samples based on specific criteria
    
    Parameters:
    -----------
    events : awkward.Array or dict-like object
        Events from uproot containing the MC sample
    track_type : str
        Track type, either 'LL' (Long-Long) or 'DD' (Downstream-Downstream)
    
    Returns:
    --------
    numpy.ndarray
        Boolean mask of selected events
    dict
        Summary of the cuts applied
    """
    # Initialize mask with all True
    mask = np.ones(len(events), dtype=bool)
    
    # Track the cuts for debugging and reporting
    cuts_summary = OrderedDict()
    initial_events = len(events)
    
    # ===== p (Proton) Cuts =====
    # MC15TuneV1_ProbNNp > 0.05
    p_prob_cut = events['p_MC15TuneV1_ProbNNp'] > 0.05
    mask = mask & p_prob_cut
    cuts_summary['proton_prob_cut'] = np.sum(p_prob_cut)
    
    # ===== Λ0 Cuts =====
    
    # ΔZ > 20 mm (difference between Lambda decay vertex and primary vertex)
    delta_z = events['L0_ENDVERTEX_Z'] - events['L0_OWNPV_Z']
    delta_z_cut = delta_z > 20
    mask = mask & delta_z_cut
    cuts_summary['delta_z_cut'] = np.sum(delta_z_cut)
    
    # χ²FD > 45 (Lambda flight distance chi2)
    fd_chi2_cut = events['L0_FDCHI2_OWNPV'] > 45
    mask = mask & fd_chi2_cut
    cuts_summary['fd_chi2_cut'] = np.sum(fd_chi2_cut)
    
    # |m(pπ⁻) - 1115.6| < 6 MeV/c²
    lambda_mass_diff = np.abs(events['L0_M'] - 1115.6)
    lambda_mass_cut = lambda_mass_diff < 6
    mask = mask & lambda_mass_cut
    cuts_summary['lambda_mass_cut'] = np.sum(lambda_mass_cut)
    
    # Lp_MC15TuneV1_ProbNNp > 0.2
    lp_prob_cut = events['Lp_MC15TuneV1_ProbNNp'] > 0.2
    mask = mask & lp_prob_cut
    cuts_summary['lp_prob_cut'] = np.sum(lp_prob_cut)
    
    # K_s veto cut removed as requested
    
    # ===== B⁺ Cuts =====
    
    # pT > 3000 MeV/c
    b_pt_cut = events['Bu_PT'] > 3000
    mask = mask & b_pt_cut
    cuts_summary['b_pt_cut'] = np.sum(b_pt_cut)
    
    # χ²DTF < 30 & Converged (DTF = Decay Tree Fitter)
    dtf_chi2 = events['Bu_DTF_chi2']
    dtf_chi2_cut = (dtf_chi2 < 30) 
    mask = mask & dtf_chi2_cut
    cuts_summary['dtf_chi2_cut'] = np.sum(dtf_chi2_cut)
    
    # χ²IP < 10 (Impact Parameter Chi2)
    ip_chi2_cut = events['Bu_IPCHI2_OWNPV'] < 10
    mask = mask & ip_chi2_cut
    cuts_summary['ip_chi2_cut'] = np.sum(ip_chi2_cut)
    
    # χ²FD > 175 (Flight Distance Chi2)
    b_fd_chi2_cut = events['Bu_FDCHI2_OWNPV'] > 175
    mask = mask & b_fd_chi2_cut
    cuts_summary['b_fd_chi2_cut'] = np.sum(b_fd_chi2_cut)
    
    # Print selection summary
    selected_events = np.sum(mask)
    print(f"Selection summary for {track_type} sample:")
    print(f"Initial events: {initial_events}")
    for cut_name, cut_count in cuts_summary.items():
        print(f"  {cut_name}: {cut_count} / {initial_events} ({cut_count/initial_events:.2%})")
    print(f"Final selected events: {selected_events} / {initial_events} ({selected_events/initial_events:.2%})")
    
    return mask, cuts_summary

def apply_cuts_to_samples(mc_ll, mc_dd):
    """
    Apply selection cuts to both LL and DD samples
    
    Parameters:
    -----------
    mc_ll : awkward.Array or dict-like object
        Long-Long track type MC sample
    mc_dd : awkward.Array or dict-like object
        Downstream-Downstream track type MC sample
    
    Returns:
    --------
    tuple
        (mc_ll_mask, mc_dd_mask, ll_cuts_summary, dd_cuts_summary)
    """
    # Apply cuts to LL sample
    ll_mask, ll_cuts_summary = apply_selection_cuts(mc_ll, track_type='LL')
    
    # Apply cuts to DD sample
    dd_mask, dd_cuts_summary = apply_selection_cuts(mc_dd, track_type='DD')
    
    # Print comparison between LL and DD
    ll_total = len(mc_ll)
    dd_total = len(mc_dd)
    ll_selected = np.sum(ll_mask)
    dd_selected = np.sum(dd_mask)
    
    print("\nComparison between LL and DD selection efficiency:")
    print(f"LL: {ll_selected}/{ll_total} ({ll_selected/ll_total:.2%})")
    print(f"DD: {dd_selected}/{dd_total} ({dd_selected/dd_total:.2%})")
    
    return ll_mask, dd_mask, ll_cuts_summary, dd_cuts_summary

def apply_mask_to_data(events, mask):
    """
    Apply a boolean mask to event data
    
    Parameters:
    -----------
    events : awkward.Array or dict-like object
        Event data
    mask : numpy.ndarray
        Boolean mask to apply
    
    Returns:
    --------
    awkward.Array or dict-like object
        Selected events
    """
    if hasattr(events, 'mask'):
        # For awkward arrays
        return events[mask]
    else:
        # For dictionary-like objects
        selected = {}
        for key, array in events.items():
            selected[key] = array[mask]
        return selected

# Plot function removed as requested


# Usage example:
"""
# Apply the cuts to get the selection masks
ll_mask, dd_mask, ll_summary, dd_summary = apply_cuts_to_samples(mc_ll, mc_dd)

# Apply the masks to get the selected events
mc_ll_selected = apply_mask_to_data(mc_ll, ll_mask)
mc_dd_selected = apply_mask_to_data(mc_dd, dd_mask)

# Cut efficiency plotting removed as requested

# Now you can proceed with analysis using the selected samples
# Further operations with mc_ll_selected and mc_dd_selected...
"""

'\n# Apply the cuts to get the selection masks\nll_mask, dd_mask, ll_summary, dd_summary = apply_cuts_to_samples(mc_ll, mc_dd)\n\n# Apply the masks to get the selected events\nmc_ll_selected = apply_mask_to_data(mc_ll, ll_mask)\nmc_dd_selected = apply_mask_to_data(mc_dd, dd_mask)\n\n# Cut efficiency plotting removed as requested\n\n# Now you can proceed with analysis using the selected samples\n# Further operations with mc_ll_selected and mc_dd_selected...\n'

In [5]:
selected_ll, selected_dd, ll_summary, dd_summary = apply_cuts_to_samples(_mc_ll, _mc_dd)

Selection summary for LL sample:
Initial events: 30616
  proton_prob_cut: 30526 / 30616 (99.71%)
  delta_z_cut: 29768 / 30616 (97.23%)
  fd_chi2_cut: 30440 / 30616 (99.43%)
  lambda_mass_cut: 29860 / 30616 (97.53%)
  lp_prob_cut: 29837 / 30616 (97.46%)
  b_pt_cut: 29978 / 30616 (97.92%)
  dtf_chi2_cut: 29693 / 30616 (96.99%)
  ip_chi2_cut: 30385 / 30616 (99.25%)
  b_fd_chi2_cut: 29268 / 30616 (95.60%)
Final selected events: 25377 / 30616 (82.89%)
Selection summary for DD sample:
Initial events: 77067
  proton_prob_cut: 76856 / 77067 (99.73%)
  delta_z_cut: 77025 / 77067 (99.95%)
  fd_chi2_cut: 72485 / 77067 (94.05%)
  lambda_mass_cut: 73883 / 77067 (95.87%)
  lp_prob_cut: 73492 / 77067 (95.36%)
  b_pt_cut: 76641 / 77067 (99.45%)
  dtf_chi2_cut: 74596 / 77067 (96.79%)
  ip_chi2_cut: 76452 / 77067 (99.20%)
  b_fd_chi2_cut: 73190 / 77067 (94.97%)
Final selected events: 60356 / 77067 (78.32%)

Comparison between LL and DD selection efficiency:
LL: 25377/30616 (82.89%)
DD: 60356/77067 (78.3

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak

def plot_selection_variables_separate(mc_ll, mc_dd, output_prefix="selection_variables"):
    """
    Plot selection variables for LL and DD samples in separate files
    
    Parameters:
    -----------
    mc_ll : awkward.Array
        Long-Long track type MC sample
    mc_dd : awkward.Array
        Downstream-Downstream track type MC sample
    output_prefix : str
        Prefix for output files
    """
    # Define variables to plot and their properties
    variables = [
        {
            'name': 'p_MC15TuneV1_ProbNNp',
            'label': 'p_MC15TuneV1_ProbNNp',
            'cut_value': 0.05,
            'cut_type': '>'
        },
        {
            'name': 'L0_FDCHI2_OWNPV',
            'label': 'Lambda χ²FD',
            'cut_value': 45,
            'cut_type': '>'
        },
        {
            'name': 'Lp_MC15TuneV1_ProbNNp',
            'label': 'Lp_MC15TuneV1_ProbNNp',
            'cut_value': 0.2,
            'cut_type': '>'
        },
        {
            'name': 'Bu_PT',
            'label': 'B+ pT [MeV/c]',
            'cut_value': 3000,
            'cut_type': '>'
        },
        {
            'name': 'Bu_IPCHI2_OWNPV',
            'label': 'B+ χ²IP',
            'cut_value': 10,
            'cut_type': '<'
        },
        {
            'name': 'Bu_FDCHI2_OWNPV',
            'label': 'B+ χ²FD',
            'cut_value': 175,
            'cut_type': '>'
        },
        {
            'name': 'h1_ProbNNk',
            'label': 'h1_ProbNNk',
            'cut_value': 0.2,
            'cut_type': '>'
        },
        {
            'name': 'h2_ProbNNk',
            'label': 'h2_ProbNNk',
            'cut_value': 0.2,
            'cut_type': '>'
        },
        # We'll handle Bu_DTF_chi2 separately
    ]
    
    # Add delta_z calculation separately
    delta_z = {
        'name': 'delta_z',
        'label': 'ΔZ [mm]',
        'cut_value': 20,
        'cut_type': '>'
    }
    
    # Add Bu_DTF_chi2 separately with special handling
    bu_dtf_chi2 = {
        'name': 'Bu_DTF_chi2',
        'label': 'B+ χ²DTF',
        'cut_value': 30,
        'cut_type': '<'
    }
    
    # Add KK product separately
    kk_product = {
        'name': 'kk_product',
        'label': 'h1_ProbNNk × h2_ProbNNk',
        'cut_value': 0.2,
        'cut_type': '>'
    }
    
    # Create figure and axes for LL
    n_vars = len(variables) + 3  # +3 for delta_z, Bu_DTF_chi2, and kk_product
    n_cols = 3
    n_rows = (n_vars + n_cols - 1) // n_cols
    
    # Create separate figures for LL and DD
    fig_ll, axes_ll = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes_ll = axes_ll.flatten()
    
    fig_dd, axes_dd = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes_dd = axes_dd.flatten()
    
    # Extract the delta_z variable
    try:
        ll_delta_z = np.array(ak.to_numpy(mc_ll['L0_ENDVERTEX_Z']) - ak.to_numpy(mc_ll['L0_OWNPV_Z']))
        dd_delta_z = np.array(ak.to_numpy(mc_dd['L0_ENDVERTEX_Z']) - ak.to_numpy(mc_dd['L0_OWNPV_Z']))
        
        # Process delta_z 
        process_variable_single(axes_ll[0], ll_delta_z, delta_z, "LL")
        process_variable_single(axes_dd[0], dd_delta_z, delta_z, "DD")
        
    except Exception as e:
        print(f"Error processing delta_z: {e}")
        axes_ll[0].text(0.5, 0.5, f"Error processing delta_z", ha='center', va='center', transform=axes_ll[0].transAxes)
        axes_dd[0].text(0.5, 0.5, f"Error processing delta_z", ha='center', va='center', transform=axes_dd[0].transAxes)
    
    # Process each standard variable
    for i, var_info in enumerate(variables):
        try:
            var_name = var_info['name']
            
            # Convert to numpy arrays to avoid awkward array issues
            try:
                ll_data = np.array(ak.to_numpy(mc_ll[var_name]))
                dd_data = np.array(ak.to_numpy(mc_dd[var_name]))
                
                # Process the variable for each plot separately
                process_variable_single(axes_ll[i+1], ll_data, var_info, "LL")
                process_variable_single(axes_dd[i+1], dd_data, var_info, "DD")
                
            except Exception as e:
                print(f"Error extracting {var_name}: {e}")
                axes_ll[i+1].text(0.5, 0.5, f"Error extracting {var_name}", 
                           ha='center', va='center', transform=axes_ll[i+1].transAxes)
                axes_dd[i+1].text(0.5, 0.5, f"Error extracting {var_name}", 
                           ha='center', va='center', transform=axes_dd[i+1].transAxes)
                
        except Exception as e:
            print(f"Error processing variable {i}: {e}")
            if i+1 < len(axes_ll):
                axes_ll[i+1].text(0.5, 0.5, f"Error in processing", 
                           ha='center', va='center', transform=axes_ll[i+1].transAxes)
                axes_dd[i+1].text(0.5, 0.5, f"Error in processing", 
                           ha='center', va='center', transform=axes_dd[i+1].transAxes)
    
    # Special handling for Bu_DTF_chi2
    try:
        # Try to get the first element from potentially irregular array structure
        ll_dtf_chi2 = []
        dd_dtf_chi2 = []
        
        # Handle irregular arrays by getting first element
        for item in mc_ll["Bu_DTF_chi2"]:
            if ak.count(item) > 0:  # Check if the array is not empty
                ll_dtf_chi2.append(float(item[0]))  # Take the first element
            else:
                ll_dtf_chi2.append(np.nan)  # Use NaN for empty arrays
                
        for item in mc_dd["Bu_DTF_chi2"]:
            if ak.count(item) > 0:
                dd_dtf_chi2.append(float(item[0]))
            else:
                dd_dtf_chi2.append(np.nan)
        
        # Convert to numpy arrays and remove NaN values
        ll_dtf_chi2 = np.array(ll_dtf_chi2)
        dd_dtf_chi2 = np.array(dd_dtf_chi2)
        
        ll_dtf_chi2 = ll_dtf_chi2[~np.isnan(ll_dtf_chi2)]
        dd_dtf_chi2 = dd_dtf_chi2[~np.isnan(dd_dtf_chi2)]
        
        # Process Bu_DTF_chi2
        process_variable_single(axes_ll[len(variables)+1], ll_dtf_chi2, bu_dtf_chi2, "LL")
        process_variable_single(axes_dd[len(variables)+1], dd_dtf_chi2, bu_dtf_chi2, "DD")
        
    except Exception as e:
        print(f"Error processing Bu_DTF_chi2: {e}")
        axes_ll[len(variables)+1].text(0.5, 0.5, f"Error processing Bu_DTF_chi2: {e}", 
                       ha='center', va='center', transform=axes_ll[len(variables)+1].transAxes)
        axes_dd[len(variables)+1].text(0.5, 0.5, f"Error processing Bu_DTF_chi2: {e}", 
                       ha='center', va='center', transform=axes_dd[len(variables)+1].transAxes)
    
    # Special handling for KK product
    try:
        # Get the individual kaon ID probabilities
        ll_h1_probnnk = np.array(ak.to_numpy(mc_ll['h1_ProbNNk']))
        ll_h2_probnnk = np.array(ak.to_numpy(mc_ll['h2_ProbNNk']))
        dd_h1_probnnk = np.array(ak.to_numpy(mc_dd['h1_ProbNNk']))
        dd_h2_probnnk = np.array(ak.to_numpy(mc_dd['h2_ProbNNk']))
        
        # Calculate the product
        ll_kk_product = ll_h1_probnnk * ll_h2_probnnk
        dd_kk_product = dd_h1_probnnk * dd_h2_probnnk
        
        # Process KK product
        process_variable_single(axes_ll[len(variables)+2], ll_kk_product, kk_product, "LL")
        process_variable_single(axes_dd[len(variables)+2], dd_kk_product, kk_product, "DD")
        
    except Exception as e:
        print(f"Error processing KK product: {e}")
        axes_ll[len(variables)+2].text(0.5, 0.5, f"Error processing KK product: {e}", 
                       ha='center', va='center', transform=axes_ll[len(variables)+2].transAxes)
        axes_dd[len(variables)+2].text(0.5, 0.5, f"Error processing KK product: {e}", 
                       ha='center', va='center', transform=axes_dd[len(variables)+2].transAxes)
        
    except Exception as e:
        print(f"Error processing Bu_DTF_chi2: {e}")
        axes_ll[len(variables)+1].text(0.5, 0.5, f"Error processing Bu_DTF_chi2: {e}", 
                       ha='center', va='center', transform=axes_ll[len(variables)+1].transAxes)
        axes_dd[len(variables)+1].text(0.5, 0.5, f"Error processing Bu_DTF_chi2: {e}", 
                       ha='center', va='center', transform=axes_dd[len(variables)+1].transAxes)
    
    # Handle any unused axes
    for i in range(n_vars, len(axes_ll)):
        axes_ll[i].set_visible(False)
        axes_dd[i].set_visible(False)
    
    # Add overall titles
    fig_ll.suptitle("Selection Variables - LL Sample", fontsize=16)
    fig_dd.suptitle("Selection Variables - DD Sample", fontsize=16)
    
    # Adjust layout
    fig_ll.tight_layout()
    plt.figure(fig_ll.number)
    plt.subplots_adjust(top=0.95)
    
    fig_dd.tight_layout()
    plt.figure(fig_dd.number)
    plt.subplots_adjust(top=0.95)
    
    # Save figures
    ll_filename = f"{output_prefix}_LL_mc.pdf"
    dd_filename = f"{output_prefix}_DD_mc.pdf"
    
    plt.figure(fig_ll.number)
    plt.savefig(ll_filename, dpi=300, bbox_inches='tight')
    
    plt.figure(fig_dd.number)
    plt.savefig(dd_filename, dpi=300, bbox_inches='tight')
    
    plt.close(fig_ll)
    plt.close(fig_dd)
    
    print(f"LL plot saved to {ll_filename}")
    print(f"DD plot saved to {dd_filename}")
    return

def process_variable_single(ax, data, var_info, sample_type):
    """
    Process and plot a single variable for a single sample
    
    Parameters:
    -----------
    ax : matplotlib.axes.Axes
        Axes to plot on
    data : numpy.ndarray
        Data from sample
    var_info : dict
        Variable information (name, label, cut_value, cut_type)
    sample_type : str
        Sample type ("LL" or "DD")
    """
    var_name = var_info['name']
    var_label = var_info['label']
    cut_value = var_info['cut_value']
    cut_type = var_info['cut_type']
    
    # Calculate pass percentage
    if cut_type == '>':
        pass_percent = (data > cut_value).sum() / len(data) * 100
    else:  # '<'
        pass_percent = (data < cut_value).sum() / len(data) * 100
    
    # Create bins for histogram
    min_val = np.min(data)
    max_val = np.max(data)
    
    # Special handling for some variables with extreme ranges
    if max_val - min_val > 1000 and 'PT' in var_name:
        # For PT variables, focus on the important range
        min_val = max(0, min_val)
        max_val = min(10000, max_val)
    elif max_val - min_val > 1000 and 'CHI2' in var_name:
        # For CHI2 variables, focus on the important range
        min_val = max(0, min_val)
        max_val = min(500, max_val)
    
    # Add padding to range
    range_padding = (max_val - min_val) * 0.1
    hist_range = (min_val - range_padding, max_val + range_padding)
    
    # Set number of bins based on data range
    if max_val - min_val > 100:
        bins = 50
    else:
        bins = 30
    
    # Create histogram with raw counts (not density)
    hist, bins = np.histogram(data, bins=bins, range=hist_range, density=False)
    
    # Calculate bin centers
    centers = (bins[:-1] + bins[1:]) / 2
    
    # Plot histograms as step plots (cleaner than bars)
    color = 'blue' if sample_type == 'LL' else 'green'
    ax.step(centers, hist, where='mid', color=color, linewidth=2)
    
    # Add vertical line at cut value
    ymin, ymax = 0, np.max(hist) * 1.1
    ax.set_ylim(ymin, ymax)
    ax.vlines(cut_value, ymin, ymax, colors='red', linestyles='dashed', linewidth=2)
    
    # Add cut text at the top of the graph
    ax.text(
        0.5, 
        0.95, 
        f"Cut: {cut_type} {cut_value} ({pass_percent:.1f}% pass)", 
        transform=ax.transAxes, 
        verticalalignment='top', 
        horizontalalignment='center',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
    )
    
    # Set title and labels
    ax.set_title(f"{var_label} - {sample_type}")
    ax.set_xlabel(var_label)
    ax.set_ylabel('Events')
    
    return

# Example usage:
"""
# Plot all selection variables for LL and DD samples separately
plot_selection_variables_separate(mc_ll, mc_dd, output_prefix="selection_variables")
"""

'\n# Plot all selection variables for LL and DD samples separately\nplot_selection_variables_separate(mc_ll, mc_dd, output_prefix="selection_variables")\n'

In [15]:
mc_ll = _mc_ll
mc_dd = _mc_dd

In [ ]:
plot_selection_variables_separate(mc_ll, mc_dd, output_prefix="selection_variables")